# MDA: Post-procesamiento

Este notebook describe los métodos de post-procesamiento que fueron aplicados sobre los resultados de las predicciones del modelo de U-Net.

## Poligonizado

La siguiente función aplica una rutina de poligonización sobre los resultados de la predicción del modelo y genera un archivo vectorial en formato GeoPackage (GPKG). La rutina aplica `gdal_polygonize.py` a cada chip resultado generando un GPKG para cada chip, y luego une todos estos archivos en uno solo, de manera eficiente.

Antes de unirlos también aplica un umbral sobre los valores de los rásteres, que en este caso representan la probabilidad (valores entre 0 y 1).

In [1]:
from meduy.postprocess.polygonize import polygonize

In [ ]:
input_path = "data/results/predict/"
output_path = "data/results/predict.gpkg"

polygonize(threshold=0.4,
           input_dir=input_dir,
           output=output_path)

## Filtrar por área mínima

Como último paso, filtramos aquellos polígonos de area inferior a 300 m², dado que consideramos que son falsos positivos. Para esto utilizamos `ogr2ogr` y una consulta SQL. El resultado se guarda en `data/results/predict_300.gpkg`

In [ ]:
min_area = 300
input_path = "data/results/predict.gpkg"
output_path = "data/results/predict_300.gpkg"

In [ ]:
!ogr2ogr \
    -s_srs EPSG:5382 \
    -t_srs EPSG:5382 \
    -f "GPKG" \
    -sql "SELECT * FROM results m WHERE ST_Area(geom) > $min_area" \
    -dialect SQLITE \
    -nln results \
    $input_path
    $output_path